<a href="https://colab.research.google.com/github/04-AhmadThariqRamadhan/Tugas7-movie-recommendations/blob/main/Tugas_7_movie_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=2f5b789b8acde0dc18cdcc3b58f94d74dcbced74b57dad2613dc3ac953e69f7b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [45]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

In [44]:
import math
spark = SparkSession.builder.master("local[*]").getOrCreate()

class Rating:
    def __init__(self, userId, movieId, rating, timestamp):
        self.userId = int(userId)
        self.movieId = int(movieId)
        self.rating = float(rating)
        self.timestamp = float(timestamp)

def parseRating(str):
    fields = str.split(",")
    assert(len(fields) == 4)
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]), float(fields[3]))

In [46]:
ratings = spark.read.csv("/content/drive/MyDrive/BIG DATA/ml-latest-small/ratings.csv", header=True, inferSchema=True)
# Check if everything is ok
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [22]:
# Split data into 80% train 20% test
training, test = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

model = als.fit(training)
model.write().overwrite().save("rama_model")

# Prepare the recommendations
predictions = model.transform(test)
squaredErrors = predictions.withColumn("squaredError", (col("rating") - col("prediction")) ** 2)
mse = predictions.withColumn("diff", col("rating") - col("prediction")).select((col("diff") ** 2).alias("squared_diff")).filter(~col("squared_diff").isNull()).agg({"squared_diff": "sum"}).collect()[0][0]

predictions.show(10)

# Menyimpan Hasil Prediksi
predictions.write.format("csv").save("ml-predictions.csv")

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|   1127|   4.0|964982513|   4.53635|
|     1|    296|   3.0|964982967| 4.0607276|
|     1|    596|   5.0|964982838|  3.936464|
|     1|    500|   3.0|964981208| 3.7870524|
|     1|    163|   5.0|964983650|  4.619319|
|     1|   1222|   5.0|964981909| 4.6925797|
|     1|    733|   4.0|964982400| 3.7069018|
|     1|    527|   5.0|964984002|  4.608643|
|     1|   1213|   5.0|964982951|  5.102218|
|     1|    608|   5.0|964982931| 4.5389333|
+------+-------+------+---------+----------+
only showing top 10 rows

